In [8]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from sklearn.preprocessing import LabelEncoder

# 1. 주차장 데이터 불러오기
df = pd.read_csv(r'src/RF_parksdf_holi.csv')

# 시간별 운영여부 컬럼 추가
for hour in range(24):
    df[f'is_open_at_{hour}'] = (
        ((df['Holiday_start'] <= hour) & (df['Holiday_end'] > hour)) |
        ((df['Holiday_start'] > df['Holiday_end']) & 
         ((hour >= df['Holiday_start']) | (hour < df['Holiday_end'])))
    ).astype(int)

# 2. geometry 변환
geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]
parking_gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")

# 3. 용도지역 폴리곤 불러오기
zone = gpd.read_file(r"src/UQ111_용도지역(도시지역)/shp파일/UPIS_C_UQ111.shp")
if zone.crs != parking_gdf.crs:
    parking_gdf = parking_gdf.to_crs(zone.crs)

# 4. 공간조인
joined = gpd.sjoin(parking_gdf, zone, how='left', predicate='within')
joined = joined.rename(columns={'DGM_NM': 'zone_name'})  # 실제 컬럼명 확인 필요

# 5. 대분류 분류 함수
def classify_zone(name):
    if pd.isnull(name): return '미분류'
    if '주거' in name: return '주거지역'
    elif '상업' in name: return '상업지역'
    elif '공업' in name or '준공업' in name or '중공업' in name: return '공업지역'
    elif '녹지' in name: return '녹지지역'
    else: return '기타'

joined['zoning'] = joined['zone_name'].apply(classify_zone)

# 6. address별로 여러 용도지역, 대분류가 있으면 모두 +로 합쳐서 하나의 값으로
grouped = (
    joined.groupby('address')
    .agg({
        'zone_name': lambda x: '+'.join(sorted(set(str(i) for i in x if pd.notnull(i)))),
        'zoning':    lambda x: '+'.join(sorted(set(str(i) for i in x if pd.notnull(i))))
    })
    .reset_index()
)

# 7. 원래 데이터와 merge
df = df.merge(grouped, on='address', how='left')

# 9. 결과 확인
print(df[['address', 'zoning']].head())

# 10. 저장
df.to_csv('src/RF_parksdf_holi_zone.csv', index=False, encoding='utf-8-sig')
print(df.head())


          address     zoning
0   강남구 개포동 126-2  녹지지역+주거지역
1  강남구 개포동 1266-0       주거지역
2  강남구 개포동 1273-0       주거지역
3    강남구 개포동 13-2       주거지역
4   강남구 개포동 183-2       주거지역
          address parking_type  operation_type  total_parking_spaces  \
0   강남구 개포동 126-2           NW               1                 132.0   
1  강남구 개포동 1266-0           NW               1                  97.0   
2  강남구 개포동 1273-0           NW               1                 194.0   
3    강남구 개포동 13-2           NW               1                 168.0   
4   강남구 개포동 183-2           NW               1                 149.0   

   base_parking_fee  base_parking_time  additional_unit_fee  \
0             200.0                5.0                200.0   
1             200.0                5.0                200.0   
2             200.0                5.0                200.0   
3             100.0                5.0                100.0   
4             200.0                5.0                200.0   

   additional_

In [9]:
# # column drop
# df = pd.read_csv(r'src/RF_parksdf_holi_zone.csv') 
# df.drop(['zone_name', 'zoning'], axis=1, inplace=True)
# df.to_csv('src/RF_parksdf_holi_zone.csv', index=False, encoding='utf-8-sig')

In [10]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# 1. 데이터 읽기
dong_df = pd.read_csv('src/dongParks_Holiday.csv')
zoning_df = pd.read_csv('행정동별_최종우세지역.csv')

# 2. dict로 만들어 mapping
zoning_dict = zoning_df.set_index('ADM_NM')['최종우세지역'].to_dict()

# 3. dong 컬럼 기준으로 zoning 붙이기
dong_df['zoning'] = dong_df['dong'].map(zoning_dict)

# 4. null 값을 '미분류'로 채우기
dong_df['zoning'] = dong_df['zoning'].fillna('미분류')


# 6. 결과 저장/확인
dong_df.to_csv('src/dongParks_Holiday_zone.csv', index=False, encoding='utf-8-sig')
print(dong_df[['dong', 'zoning']].head())

   dong     zoning
0  가락본동       주거지역
1  가리봉동       주거지역
2   가산동       공업지역
3  가양1동  공업지역+녹지지역
4  가양2동       녹지지역


In [11]:
parks_df_zone = pd.read_csv('src/RF_parksdf_holi_zone.csv')
dong_zone = pd.read_csv('src/dongParks_Holiday_zone.csv')

zoning_set = set(parks_df_zone['zoning']) | set(dong_zone['zoning'])
all_zonings = pd.Series(sorted(zoning_set))

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(all_zonings)

parks_df_zone['zoning_encoded'] = le.transform(parks_df_zone['zoning'])
dong_zone['zoning_encoded'] = le.transform(dong_zone['zoning'])

parks_df_zone.to_csv('src/RF_parksdf_holi_zone.csv', index=False, encoding='utf-8-sig')
dong_zone.to_csv('src/dongParks_Holiday_zone.csv', index=False, encoding='utf-8-sig')

In [12]:
print(parks_df_zone['zoning_encoded'].value_counts())
print('고유값:', parks_df_zone['zoning_encoded'].unique())
print('고유값 개수:', parks_df_zone['zoning_encoded'].nunique())

print(dong_zone['zoning_encoded'].value_counts())
print('고유값:', dong_zone['zoning_encoded'].unique())
print('고유값 개수:', dong_zone['zoning_encoded'].nunique())

zoning_encoded
10    1161
8      248
0      121
5       67
4       13
6        7
9        4
7        3
3        1
Name: count, dtype: int64
고유값: [ 6 10  8  5  0  7  3  9  4]
고유값 개수: 9
zoning_encoded
10    238
5      47
0       8
8       6
6       4
1       3
12      3
11      1
9       1
7       1
2       1
Name: count, dtype: int64
고유값: [10  0  1  5 11 12  8  9  6  7  2]
고유값 개수: 11


In [13]:
# column drop
df = pd.read_csv('src/RF_parksdf_holi_zone.csv') 
df.drop(['zone_name', 'zoning'], axis=1, inplace=True)
df.to_csv('src/RF_parksdf_holi_zone.csv', index=False, encoding='utf-8-sig')

In [14]:
# column drop
df = pd.read_csv('src/dongParks_Holiday_zone.csv') 
df.drop(['zoning'], axis=1, inplace=True)
df.to_csv('src/dongParks_Holiday_zone.csv', index=False, encoding='utf-8-sig')